In [1]:
import os, json
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt

from skimage.morphology import skeletonize_3d
from skimage.measure import label
from ipywidgets import interact, IntSlider, FloatSlider

In [2]:
# ===== Input =====
DICOM_DIR = r'/Users/tkimura/Desktop/Image data/5396641 IPMN/DICOM-A/ScalarVolume_22'  # ★ change

# ===== Bone threshold (only for hard bone mask; we won't overuse it) =====
BONE_HU = 350   # keep as-is for now

# ===== Thin vessels (current) =====
SIGMAS_MM_THIN = (0.5, 0.8, 1.2, 1.8, 2.6)
THR_PERCENTILE_THIN = 98.5
MIN_VOX_THIN = 300

# ===== Thick vessels (NEW) =====
# 太い血管（大動脈など）を拾うには、σを大きくする必要があります
SIGMAS_MM_THICK = (2.0, 3.0, 4.5, 6.5)   # 太い本幹向け
THR_PERCENTILE_THICK = 97.0             # 太い血管は反応が広くなるので、少し緩め
MIN_VOX_THICK = 2000                    # 太い血管は塊が大きいので、強めにゴミ除去

# ===== Seed (for aorta / main artery) =====
# ★まずは手動で。大動脈の内腔中心を (z,y,x) で指定
SEED_ZYX = (94, 260, 260)  # ←今の表示z=94に合わせて仮置き。後で調整

print("DICOM_DIR:", DICOM_DIR)
print("THIN:", SIGMAS_MM_THIN, THR_PERCENTILE_THIN, MIN_VOX_THIN)
print("THICK:", SIGMAS_MM_THICK, THR_PERCENTILE_THICK, MIN_VOX_THICK)
print("SEED_ZYX:", SEED_ZYX)


DICOM_DIR: /Users/tkimura/Desktop/Image data/5396641 IPMN/DICOM-A/ScalarVolume_22
THIN: (0.5, 0.8, 1.2, 1.8, 2.6) 98.5 300
THICK: (2.0, 3.0, 4.5, 6.5) 97.0 2000
SEED_ZYX: (94, 260, 260)


In [3]:
#Cell2
reader = sitk.ImageSeriesReader()
series_ids = reader.GetGDCMSeriesIDs(DICOM_DIR)
if not series_ids:
    raise RuntimeError("DICOM series not found. Check DICOM_DIR.")

# まずは1シリーズ目（必要ならここを選別）
files = reader.GetGDCMSeriesFileNames(DICOM_DIR, series_ids[0])
reader.SetFileNames(files)
img = reader.Execute()  # sitk.Image

spacing = img.GetSpacing()   # (sx, sy, sz) mm
origin  = img.GetOrigin()
direction = img.GetDirection()

ct_np = sitk.GetArrayFromImage(img)  # z,y,x

print("Size(x,y,z):", img.GetSize())
print("Spacing(mm):", spacing)
print("CT array shape (z,y,x):", ct_np.shape)
print("HU stats:", float(np.min(ct_np)), float(np.median(ct_np)), float(np.max(ct_np)))


Size(x,y,z): (512, 512, 209)
Spacing(mm): (0.625, 0.625, 1.2500000000000004)
CT array shape (z,y,x): (209, 512, 512)
HU stats: -3024.0 -911.0 2248.0


In [4]:
# ✅ Cell 2.5: Body mask (remove air & table) — VERY IMPORTANT

import numpy as np
import SimpleITK as sitk
from scipy.ndimage import binary_fill_holes

# --- parameters ---
AIR_HU = -300   # 空気と体内の境界
MIN_BODY_VOX = 50000  # 体として残す最小サイズ

# 1) threshold to separate body from air
body0 = (ct_np > AIR_HU)

# 2) fill holes (lung, bowel gas)
body1 = binary_fill_holes(body0)

# 3) keep largest connected component (the body)
from skimage.measure import label
lab = label(body1, connectivity=2)
counts = np.bincount(lab.ravel())
counts[0] = 0
body_id = np.argmax(counts)

body_np = (lab == body_id)

print("Body voxels:", int(body_np.sum()))

# 4) visualize quick sanity
coords = np.argwhere(body_np)
print("Body bbox (z,y,x):",
      tuple(coords.min(axis=0)),
      "->",
      tuple(coords.max(axis=0)))

Body voxels: 18831016
Body bbox (z,y,x): (0, 105, 8) -> (208, 423, 504)


In [5]:
# ✅ Cell 3 (revised): Bone removal + body ROI restriction

BONE_HU = 350

# bone removal
bone = (ct_np > BONE_HU)
soft_np = ct_np.copy()
soft_np[bone] = 0

# apply body mask
soft_np[~body_np] = 0

print("Soft tissue voxels after body ROI:", int((soft_np != 0).sum()))


Soft tissue voxels after body ROI: 18209177


In [6]:
# ✅ Cell 4: Frangi vesselness for THIN + THICK

import numpy as np
from skimage.filters import frangi

def frangi_vesselness(ct_np_zyx, sigmas_mm, spacing_mm):
    sx, sy, sz = spacing_mm
    ref = float(min(sx, sy))  # in-plane spacing 기준
    sigmas_px = [float(s / ref) for s in sigmas_mm]

    print("Frangi sigmas (mm):", sigmas_mm)
    print("Frangi sigmas (px):", sigmas_px)

    vol = ct_np_zyx.astype(np.float32)
    vmin, vmax = np.percentile(vol, [1, 99])
    vol = np.clip((vol - vmin) / (vmax - vmin + 1e-6), 0.0, 1.0)

    vness = frangi(
        vol,
        sigmas=sigmas_px,
        alpha=0.5,
        beta=0.5,
        gamma=15,
        black_ridges=False
    )
    return vness

# --- run thin / thick ---
vness_thin_np  = frangi_vesselness(soft_np, SIGMAS_MM_THIN, spacing)
vness_thick_np = frangi_vesselness(soft_np, SIGMAS_MM_THICK, spacing)

# --- quick stats ---
def stat(name, v):
    pos = v[v > 0]
    print(f"\n[{name}] shape:", v.shape, "pos count:", int(pos.size))
    if pos.size:
        for p in [95, 97, 98, 98.5, 99]:
            print(f"  percentile {p:>4}:", float(np.percentile(pos, p)))
        print("  min/median/max:", float(np.min(pos)), float(np.median(pos)), float(np.max(pos)))

stat("thin", vness_thin_np)
stat("thick", vness_thick_np)


Frangi sigmas (mm): (0.5, 0.8, 1.2, 1.8, 2.6)
Frangi sigmas (px): [0.8, 1.28, 1.92, 2.88, 4.16]
Frangi sigmas (mm): (2.0, 3.0, 4.5, 6.5)
Frangi sigmas (px): [3.2, 4.8, 7.2, 10.4]

[thin] shape: (209, 512, 512) pos count: 8407066
  percentile   95: 7.428287858601834e-06
  percentile   97: 1.2905582161693014e-05
  percentile   98: 1.8789040768751882e-05
  percentile 98.5: 2.365473087593261e-05
  percentile   99: 3.1391267839353125e-05
  min/median/max: 1.401298464324817e-45 1.7586376088729594e-07 0.000438924937043339

[thick] shape: (209, 512, 512) pos count: 3741946
  percentile   95: 1.4600543352116802e-07
  percentile   97: 1.9539668159040949e-07
  percentile   98: 2.420042221729092e-07
  percentile 98.5: 2.7955457966299887e-07
  percentile   99: 3.384412210039039e-07
  min/median/max: 1.401298464324817e-45 1.470448385276768e-08 1.6663698261254467e-06


In [7]:
# ✅ Cell 5 (robust): Build THIN/THICK masks + auto-find seed for THICK if needed

import numpy as np
import SimpleITK as sitk
from skimage.measure import label

def build_mask_from_vness(vness_np, thr_percentile, min_vox):
    pos = vness_np[vness_np > 0]
    if pos.size == 0:
        raise RuntimeError("No positive vesselness values.")
    thr = float(np.percentile(pos, thr_percentile))
    m0 = (vness_np >= thr).astype(np.uint8)

    lab = label(m0, connectivity=2)
    counts = np.bincount(lab.ravel())
    counts[0] = 0
    keep = np.where(counts >= int(min_vox))[0]
    m1 = np.isin(lab, keep).astype(np.uint8)
    return m1, thr

def seed_component(mask_np, seed_zyx):
    lab = label((mask_np > 0).astype(np.uint8), connectivity=2)
    z, y, x = map(int, seed_zyx)
    if not (0 <= z < lab.shape[0] and 0 <= y < lab.shape[1] and 0 <= x < lab.shape[2]):
        raise ValueError("Seed out of range.")
    seed_label = int(lab[z, y, x])
    return seed_label, (lab == seed_label).astype(np.uint8)

def auto_seed_from_ct(mask_np, ct_np, z_hint=None, center_frac=0.35, hu_percentile=99.5):
    """
    Find a seed voxel likely inside aorta:
    - restrict to central region (avoid edges/body wall)
    - choose high CT intensity voxels (contrast)
    - require mask_np == 1
    """
    Z, Y, X = mask_np.shape
    cy, cx = Y // 2, X // 2
    ry, rx = int(Y * center_frac / 2), int(X * center_frac / 2)

    y0, y1 = max(0, cy - ry), min(Y, cy + ry)
    x0, x1 = max(0, cx - rx), min(X, cx + rx)

    # choose z slice: use hint if provided, otherwise scan all
    z_list = [int(z_hint)] if (z_hint is not None and 0 <= z_hint < Z) else list(range(Z))

    best = None  # (scoreHU, z,y,x)
    for z in z_list:
        m = mask_np[z, y0:y1, x0:x1]
        if m.sum() == 0:
            continue
        a = ct_np[z, y0:y1, x0:x1].astype(np.float32)
        # threshold on HU
        hu_thr = np.percentile(a[m > 0], hu_percentile) if np.any(m > 0) else None
        if hu_thr is None:
            continue
        cand = np.argwhere((m > 0) & (a >= hu_thr))
        if cand.size == 0:
            continue
        # take the brightest candidate
        for yy, xx in cand:
            hu = float(a[yy, xx])
            zy = y0 + int(yy)
            zx = x0 + int(xx)
            if (best is None) or (hu > best[0]):
                best = (hu, z, zy, zx)

    if best is None and z_hint is not None:
        # fallback: scan all z if hint failed
        return auto_seed_from_ct(mask_np, ct_np, z_hint=None, center_frac=center_frac, hu_percentile=hu_percentile)

    return best  # None or (hu, z, y, x)

# --- THIN mask ---
mask_thin_np, thr_thin = build_mask_from_vness(vness_thin_np, THR_PERCENTILE_THIN, MIN_VOX_THIN)
print("THIN thr:", thr_thin, "vox:", int(mask_thin_np.sum()))

# --- THICK mask ---
mask_thick_raw_np, thr_thick = build_mask_from_vness(vness_thick_np, THR_PERCENTILE_THICK, MIN_VOX_THICK)
print("THICK thr:", thr_thick, "vox(raw):", int(mask_thick_raw_np.sum()))

# --- Try user seed first ---
seed_label, mask_thick_seed_np = seed_component(mask_thick_raw_np, SEED_ZYX)
print("User SEED_ZYX:", SEED_ZYX, "-> seed label:", seed_label)

if seed_label == 0:
    print("⚠️ User seed is not inside THICK mask. Auto-searching seed near center using CT intensity...")
    best = auto_seed_from_ct(mask_thick_raw_np, ct_np, z_hint=SEED_ZYX[0], center_frac=0.40, hu_percentile=99.0)
    if best is None:
        raise RuntimeError("Auto seed search failed. Try lowering THR_PERCENTILE_THICK (e.g., 96) or adjust SIGMAS_MM_THICK.")
    hu, z, y, x = best
    SEED_ZYX_AUTO = (int(z), int(y), int(x))
    print("Auto seed found (HU, z,y,x):", hu, SEED_ZYX_AUTO)

    seed_label2, mask_thick_seed_np2 = seed_component(mask_thick_raw_np, SEED_ZYX_AUTO)
    print("Auto seed label:", seed_label2)
    if seed_label2 == 0:
        raise RuntimeError("Auto seed still not inside THICK mask. Reduce THR_PERCENTILE_THICK or widen center_frac.")
    mask_thick_np = mask_thick_seed_np2
else:
    mask_thick_np = mask_thick_seed_np

print("THICK vox(seed):", int(mask_thick_np.sum()))

# --- union (final mask) ---
mask_np = ((mask_thin_np > 0) | (mask_thick_np > 0)).astype(np.uint8)
print("FINAL mask vox:", int(mask_np.sum()))

# --- build sitk mask with geometry ---
mask = sitk.GetImageFromArray(mask_np)
mask.CopyInformation(img)

coords = np.argwhere(mask_np > 0)
zmin, ymin, xmin = coords.min(axis=0)
zmax, ymax, xmax = coords.max(axis=0)
print("mask bbox (z,y,x):", (int(zmin),int(ymin),int(xmin)), "->", (int(zmax),int(ymax),int(xmax)))


THIN thr: 2.365473087593261e-05 vox: 20989
THICK thr: 1.9539668159040949e-07 vox(raw): 41344
User SEED_ZYX: (94, 260, 260) -> seed label: 0
⚠️ User seed is not inside THICK mask. Auto-searching seed near center using CT intensity...
Auto seed found (HU, z,y,x): 351.0 (94, 223, 280)
Auto seed label: 2
THICK vox(seed): 2682
FINAL mask vox: 23535
mask bbox (z,y,x): (0, 130, 64) -> (208, 387, 401)


In [8]:
# ✅ Cell 5.2: Contrast-HU mask for thick trunks (Aorta/Iliac/Splenic root) + seed-select
# Add AFTER you have body_np and ct_np available.
# Output: mask_np updated (union of previous mask_np and HU-based trunk mask)

import numpy as np
from skimage.measure import label

# ---- tune ----
HU_MIN = 180          # まず180（動脈相の幹はここより上が多い）
HU_MAX = 600          # 石灰化や骨を避けたいなら上限（600くらい）
MIN_VOX_HU = 1500     # ゴミ除去
SEED_ZYX_HU = SEED_ZYX  # 既に見つけたseedを使う（自動seedでもOK）

print("HU_MIN/HU_MAX:", HU_MIN, HU_MAX)
print("MIN_VOX_HU:", MIN_VOX_HU)
print("SEED_ZYX_HU:", SEED_ZYX_HU)

# 1) HU threshold inside body only
hu_mask = (ct_np >= HU_MIN) & (ct_np <= HU_MAX) & body_np

# 2) connected component cleanup
lab = label(hu_mask.astype(np.uint8), connectivity=2)
counts = np.bincount(lab.ravel())
counts[0] = 0
keep = np.where(counts >= int(MIN_VOX_HU))[0]
hu_mask2 = np.isin(lab, keep)

print("HU mask vox (after MIN_VOX_HU):", int(hu_mask2.sum()), "components kept:", int(len(keep)))

# 3) seed-select the component that contains SEED_ZYX_HU
z, y, x = map(int, SEED_ZYX_HU)
seed_label = int(lab[z, y, x])
print("seed label in HU-mask:", seed_label)

if seed_label == 0:
    print("⚠️ Seed is not inside HU-mask. Try lowering HU_MIN (e.g., 150) or move seed into aorta lumen.")
    hu_seed = np.zeros_like(hu_mask2, dtype=bool)
else:
    hu_seed = (lab == seed_label)

print("HU-seed vox:", int(hu_seed.sum()))

# 4) union with existing final mask
mask_np_before = mask_np.copy()
mask_np = ((mask_np > 0) | (hu_seed > 0)).astype(np.uint8)

print("mask vox before:", int(mask_np_before.sum()))
print("mask vox after :", int(mask_np.sum()))


HU_MIN/HU_MAX: 180 600
MIN_VOX_HU: 1500
SEED_ZYX_HU: (94, 260, 260)
HU mask vox (after MIN_VOX_HU): 1404517 components kept: 13
seed label in HU-mask: 0
⚠️ Seed is not inside HU-mask. Try lowering HU_MIN (e.g., 150) or move seed into aorta lumen.
HU-seed vox: 0
mask vox before: 23535
mask vox after : 23535


In [9]:
# ✅ Cell 6: Overlay check (final mask)

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider

def show_mask_overlay(z=0, ww=350, wl=50, alpha=0.5):
    a = ct_np[z].astype(np.float32)
    vmin, vmax = wl-ww/2, wl+ww/2
    a = np.clip(a, vmin, vmax)

    m = mask_np[z]
    plt.figure(figsize=(7,7))
    plt.imshow(a, cmap="gray")
    plt.imshow(np.ma.masked_where(m==0, m), cmap="autumn", alpha=alpha)
    plt.title(f"Final mask overlay z={z}")
    plt.axis("off")
    plt.show()

interact(
    show_mask_overlay,
    z=IntSlider(min=0, max=ct_np.shape[0]-1, step=1, value=94),
    ww=(100,1000,50),
    wl=(-100,300,10),
    alpha=FloatSlider(min=0.1, max=0.9, step=0.1, value=0.5)
);


interactive(children=(IntSlider(value=94, description='z', max=208), IntSlider(value=350, description='ww', ma…

In [10]:
# ✅ Cell 7 (revised): Skeletonize using new skimage API + radius (mm)

import numpy as np
import SimpleITK as sitk
from skimage.morphology import skeletonize

# 1) skeleton (new API)
skel_np = skeletonize(mask_np.astype(bool)).astype(np.uint8)
print("skeleton voxels:", int(skel_np.sum()))
if skel_np.sum() == 0:
    raise RuntimeError("Skeleton is empty. Mask may be too sparse.")

# 2) distance map for radius (mm)
mask_sitk = sitk.GetImageFromArray(mask_np.astype(np.uint8))
mask_sitk.CopyInformation(img)

dist = sitk.SignedMaurerDistanceMap(
    sitk.Cast(mask_sitk, sitk.sitkUInt8),
    insideIsPositive=True,
    squaredDistance=False,
    useImageSpacing=True
)
dist_np = sitk.GetArrayFromImage(dist)  # z,y,x in mm

rvals = dist_np[skel_np > 0]
rvals = rvals[rvals > 0]
print(
    "radius(mm) on skeleton: min / median / max =",
    float(np.min(rvals)) if rvals.size else None,
    float(np.median(rvals)) if rvals.size else None,
    float(np.max(rvals)) if rvals.size else None,
)


skeleton voxels: 5254
radius(mm) on skeleton: min / median / max = 0.625 0.625 1.25


In [11]:
# Cell8: voxel (z,y,x) -> physical mm (x,y,z) conversion

def zyx_to_mm(zyx):
    """
    Input: (z,y,x) in numpy indexing
    Output: (x_mm, y_mm, z_mm) in physical space (mm)
    """
    z, y, x = zyx
    # sitk expects (x,y,z) index as integers; we use nearest voxel for physical anchor
    ix = (int(round(x)), int(round(y)), int(round(z)))
    return np.array(img.TransformIndexToPhysicalPoint(ix), dtype=float)

shape_zyx = ct_np.shape  # (Z,Y,X)
print("shape_zyx:", shape_zyx)
print("example mm at center voxel:", zyx_to_mm((shape_zyx[0]//2, shape_zyx[1]//2, shape_zyx[2]//2)))


shape_zyx: (209, 512, 512)
example mm at center voxel: [  -3.    0. -130.]


In [12]:
# Cell9: build skeleton node list + adjacency (26-neighborhood)

import numpy as np

# skeleton voxel coordinates (z,y,x)
skel_coords = np.argwhere(skel_np > 0).astype(np.int32)
N = skel_coords.shape[0]
print("N skeleton nodes:", N)

# map voxel -> node id (use int64 key for speed)
Z, Y, X = skel_np.shape
def key_zyx(z,y,x):
    return (int(z) * (Y*X)) + (int(y) * X) + int(x)

id_map = { key_zyx(z,y,x): i for i,(z,y,x) in enumerate(skel_coords) }

# 26-neighborhood offsets (excluding 0,0,0)
nbr_offsets = [(dz,dy,dx)
               for dz in (-1,0,1)
               for dy in (-1,0,1)
               for dx in (-1,0,1)
               if not (dz==0 and dy==0 and dx==0)]

# adjacency list
adj = [[] for _ in range(N)]
deg = np.zeros(N, dtype=np.int16)

for i,(z,y,x) in enumerate(skel_coords):
    for dz,dy,dx in nbr_offsets:
        zz, yy, xx = z+dz, y+dy, x+dx
        if 0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X:
            j = id_map.get(key_zyx(zz,yy,xx), None)
            if j is not None:
                adj[i].append(j)
    deg[i] = len(adj[i])

print("degree stats: min/median/max =", int(deg.min()), int(np.median(deg)), int(deg.max()))
print("endpoints(deg==1):", int(np.sum(deg==1)), "junctions(deg>=3):", int(np.sum(deg>=3)))


N skeleton nodes: 5254
degree stats: min/median/max = 1 2 5
endpoints(deg==1): 246 junctions(deg>=3): 569


In [13]:
# Cell10: trace centerline segments split at branchpoints (junctions) and endpoints

def trace_segments(adj, deg):
    N = len(adj)
    # boundary nodes: endpoints (1) or junctions (>=3) or isolated (0)
    is_boundary = (deg != 2)

    visited_edge = set()  # store directed edges (u,v)

    segments = []

    def mark_edge(u,v):
        visited_edge.add((u,v))
        visited_edge.add((v,u))

    for u in range(N):
        if not is_boundary[u]:
            continue
        for v in adj[u]:
            if (u,v) in visited_edge:
                continue

            # start a new path u -> v -> ...
            path = [u, v]
            mark_edge(u, v)

            prev, cur = u, v
            while True:
                if is_boundary[cur]:
                    # stop at boundary (endpoint/junction)
                    break

                # cur has deg==2; continue to the neighbor that is not prev
                nbs = adj[cur]
                nxt = nbs[0] if nbs[0] != prev else nbs[1]

                if (cur, nxt) in visited_edge:
                    # already consumed
                    break

                path.append(nxt)
                mark_edge(cur, nxt)
                prev, cur = cur, nxt

            segments.append(path)

    return segments

segments = trace_segments(adj, deg)
print("segments:", len(segments))
lens = np.array([len(s) for s in segments], dtype=int)
print("segment length(nodes) min/median/max =", int(lens.min()) if len(lens) else None,
      int(np.median(lens)) if len(lens) else None,
      int(lens.max()) if len(lens) else None)


segments: 996
segment length(nodes) min/median/max = 2 3 268


In [14]:
# Cell11: convert segments -> points_mm and resample -> segments_rs (as points arrays)

import numpy as np

RESAMPLE_STEP_MM = 1.0  # お好みで
MIN_SEG_NODES = 10      # 短い枝を除去（以前のmin_seg_vox相当）

def seg_to_points_mm(seg):
    pts = np.stack([zyx_to_mm(tuple(skel_coords[v])) for v in seg], axis=0)  # (N,3) in mm
    # NaN/Inf guard
    m = np.isfinite(pts).all(axis=1)
    pts = pts[m]
    return pts

def resample_polyline_mm(pts_mm, step_mm=1.0):
    if pts_mm.shape[0] < 2:
        return pts_mm
    dif = np.diff(pts_mm, axis=0)
    seglen = np.linalg.norm(dif, axis=1)
    s = np.concatenate([[0.0], np.cumsum(seglen)])
    total = s[-1]
    if total <= 1e-9:
        return pts_mm[[0]]
    n = max(2, int(np.floor(total / step_mm)) + 1)
    t = np.linspace(0.0, total, n)
    out = np.zeros((n, 3), dtype=float)
    for k in range(3):
        out[:, k] = np.interp(t, s, pts_mm[:, k])
    return out

segments_rs = []
for seg in segments:
    if len(seg) < MIN_SEG_NODES:
        continue
    pts = seg_to_points_mm(seg)
    if pts.shape[0] < 2:
        continue
    if RESAMPLE_STEP_MM and RESAMPLE_STEP_MM > 0:
        pts = resample_polyline_mm(pts, RESAMPLE_STEP_MM)
    if pts.shape[0] >= 2:
        segments_rs.append(pts)

print("segments_rs (points arrays):", len(segments_rs))
if segments_rs:
    total_pts = sum(s.shape[0] for s in segments_rs)
    print("total points:", total_pts, "example shape:", segments_rs[0].shape)


segments_rs (points arrays): 155
total points: 4331 example shape: (95, 3)


In [15]:
# Cell12: anchor to CT volume center and export JSON

import json
from datetime import datetime

def compute_ct_center_mm_from_shape(shape_zyx):
    Nz, Ny, Nx = shape_zyx
    cz = (Nz - 1) / 2.0
    cy = (Ny - 1) / 2.0
    cx = (Nx - 1) / 2.0
    return np.array(zyx_to_mm((cz, cy, cx)), dtype=float)

ct_center_mm = compute_ct_center_mm_from_shape(shape_zyx)
print("CT volume center(mm):", ct_center_mm)

polylines_out = []
for i, pts in enumerate(segments_rs):
    pts2 = (pts - ct_center_mm).astype(float)
    polylines_out.append({
        "id": int(i),
        "n_points": int(pts2.shape[0]),
        "points_mm": pts2.tolist()
    })

out = {
    "meta": {
        "created": datetime.now().isoformat(timespec="seconds"),
        "shape_zyx": list(map(int, shape_zyx)),
        "resample_step_mm": float(RESAMPLE_STEP_MM),
        "min_seg_nodes": int(MIN_SEG_NODES),
        "note": "Each polyline is a path candidate split at branchpoints. points_mm are CT physical coords (mm) shifted so CT volume center is (0,0,0).",
        "anchor": {
            "type": "CT_volume_center",
            "ct_center_mm_original": ct_center_mm.tolist(),
            "meaning": "All points_mm shifted by -ct_center_mm_original so CT volume center becomes (0,0,0)."
        }
    },
    "polylines": polylines_out
}

OUT_PATH = "centerline_polylines_mm_centered.json"
with open(OUT_PATH, "w", encoding="utf-8") as f:
    json.dump(out, f, ensure_ascii=False, indent=2)

print("Wrote:", OUT_PATH, "polylines:", len(polylines_out))


CT volume center(mm): [  -3.    0. -130.]
Wrote: centerline_polylines_mm_centered.json polylines: 155
